# Amazon reviews K-NN

1.   Scaricare i dati disponibili a questo url: [amazon reviews](https://www.kaggle.com/datasets/kritanjalijain/amazon-reviews?resource=download) o questo [amazon review dropbox](https://www.dropbox.com/scl/fi/ucfoh391qalha3lz0bzjx/amazon_review_polarity_csv.tgz.zip?rlkey=m3a0bbp2ep4sh2qisaz0xwo1w&dl=0)


2.   Il dataset è composto da due file: train and test in csv. Ogni file contiene le seguenti informazioni
  *   polarity - 1 for negative and 2 for positive
  *   title - review heading
  *   text - review body

3.  Generare i vettori sparsi applicando il q-shingle ai dati di training con q=3.
4. Sui vettori sparsi Applicare il MinHashing LSH sul dataset di training.
5. USare il file di testing e applicare una k-nearest neighbor con i dati di testing su cui è stato applicato l'hashing. Usare k=3 e classificare l'elemento con del test set con la polarità maggiormente presente.


6. *Identificare i cluster di recensioni. Ogni cluster di recensione contiene le coppie di recensioni che hanno una similarità  > di 0.6. Da svolgere dopo l'introduzione alle network







In [1]:
import pyspark
from pyspark.mllib import *
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [2]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

25/03/29 20:46:28 WARN Utils: Your hostname, spurspc resolves to a loopback address: 127.0.1.1; using 192.168.1.46 instead (on interface wlp0s20f3)
25/03/29 20:46:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/29 20:46:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/29 20:46:30 WARN Utils: Service 'SparkUI' could not bind on port 4050. Attempting port 4051.


# Getting the datasets

In [3]:
schema = StructType([
    StructField('polarity', IntegerType(), True),
    StructField('title', StringType(), True),
    StructField('description', StringType(), True),
])
df_train = spark.read.format("csv") \
               .option("header", "false") \
               .schema(schema) \
               .load("datasets/train.csv")

# Mostra lo schema del DataFrame
df_train.printSchema()

root
 |-- polarity: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)



In [4]:
df_train.count()

3600000

In [5]:
df_test = spark.read.format("csv") \
               .option("header", "false") \
               .schema(schema) \
               .load("datasets/test.csv")

# Mostra lo schema del DataFrame
df_test.printSchema()

root
 |-- polarity: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)



In [6]:
df_test.count()

400000

# Q-shingle
$q = 3$

In [7]:
def shingle(text: str, q: int):
    shingle_set = []
    for i in range(len(text) - q + 1):
        shingle_set.append(text[i:i+q])
    return list(set(shingle_set))

shingle_udf = F.udf(shingle, ArrayType(StringType()))

q = 3
df_with_shingles = df_train.limit(10).withColumn("shingles", shingle_udf(F.col("description"), F.lit(q)))

# Mostra il DataFrame con i shingles
df_with_shingles.select('shingles').show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
distinct_shingles_df = df_with_shingles.select(F.explode(F.col('shingles')).alias('shingle')).distinct()

# Estrai i shingles distinti come lista
vocab = [row['shingle'] for row in distinct_shingles_df.collect()]

# Ordina il vocabolario
vocab = sorted(vocab)

# Funzione per applicare la one-hot encoding
def one_hot_encode(shingles, vocab):
    return [1 if shingle in shingles else 0 for shingle in vocab]

# Crea una nuova colonna 'one_hot' con i vettori one-hot
one_hot_udf = F.udf(lambda shingles: one_hot_encode(shingles, vocab), ArrayType(IntegerType()))

# Aggiungi la colonna 'one_hot' al DataFrame
df_with_one_hot = df_with_shingles.withColumn("one_hot", one_hot_udf(F.col("shingles")))

# Mostra il DataFrame con la colonna 'one_hot'
df_with_one_hot.select("one_hot").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Min-hash

In [21]:
mh = MinHashLSH(inputCol="shingles", outputCol="hashes", numHashTables=3)
model = mh.fit(df_with_shingles)

model.transform(df_with_shingles).show()

IllegalArgumentException: requirement failed: Column shingles must be of type class org.apache.spark.ml.linalg.VectorUDT:struct<type:tinyint,size:int,indices:array<int>,values:array<double>> but was actually class org.apache.spark.sql.types.ArrayType:array<string>.